In [10]:
# importing the dependencies
from itertools import product as itp
from pythtb import *
from triqs.lattice.tight_binding import TBLattice
import sympy as sp
import warnings
import numpy as np
import matplotlib.pyplot as plt

from triqs.lattice.utils import TB_from_pythTB
w90_input = w90('La2CuO4 Files', 'lco')
fermi_ev = 12.7367
w90_model = w90_input.model(zero_energy = fermi_ev, min_hopping_norm = 0.038, max_distance = None)
w90_triqs = TB_from_pythTB(w90_model)

def compareBandStructure(w90_input, fermi_ev): 
    # get band structure from wannier90
    # BLACK - the full model
    # the purpose of this function is to compare the interpolation in W90 with that in PythTB
    # returns kpts (k-points in reduced coordinates used in the interpolation in W90 code) and
    # ene (energies interpolated by W90 in the form of ene[band, kpoint])
    (w90_kpt, w90_evals) = w90_input.w90_bands_consistency()

    # get the simplified model by introducing cutoff parameters
    w90_model_038 = w90_input.model(zero_energy = fermi_ev, min_hopping_norm = 0.038, max_distance = None)
    w90_model_05 = w90_input.model(zero_energy = fermi_ev, min_hopping_norm = 0.05, max_distance = None)
    w90_model_45 = w90_input.model(zero_energy = fermi_ev, min_hopping_norm = 0.45, max_distance = None)

    # RED - the reduced model
    # solve simplified model on the same k-path as in wannier90
    # solves for eigenvalues and (optionally) eigenvectors of the TB model on a given 1D list of k-vectors
    int_evals_038 = w90_model_038.solve_all(w90_kpt)
    int_evals_05 = w90_model_05.solve_all(w90_kpt)
    int_evals_45 = w90_model_45.solve_all(w90_kpt)


    # plotting the comparison of the 2 models

    # the plotting
    fig, ax = plt.subplots(dpi = 2000)

    for i in range(w90_evals.shape[0]):
        x_axis = list(range(w90_evals.shape[1]))
        y_axis = w90_evals[i] - fermi_ev
        ax.plot(x_axis, y_axis, "k-", zorder = -100)

    # the reduced model
    for i in range(int_evals_038.shape[0]):
        ax.plot(list(range(int_evals_038.shape[1])), int_evals_038[i], "r-", zorder = -50)
        ax.plot(list(range(int_evals_05.shape[1])), int_evals_05[i], "g-", zorder = -50)
        ax.plot(list(range(int_evals_45.shape[1])), int_evals_45[i], "c-", zorder = -50)
    
    ax.axhline(y = 0, color = 'm', linestyle = '--')
    ax.set_yticks(np.arange(-2, 2.1, step = 1))

    # increasing the line width of the plotted lines
    lines = ax.lines
    for line in lines:
        line.set_linewidth(2.5)

    G = np.array([ 0.00, 0.00, 0.00])
    N = np.array([ 0.25,-0.25, 0.25])
    X = np.array([ 0.00, 0.00, 0.50])
    M = np.array([ 0.25, 0.25,-0.25])

    kpath_labels = ['G', 'N', 'X', 'G', 'M']
    idx_kpt = {}

    for name, idx in (zip(('G', 'N', 'X', 'M'), [G, N, X, M])):
        idx_kpt[name] = list(np.where((w90_kpt == idx).all(axis=1))[0])
    
    kpath_xticks = list([x[0] for x in idx_kpt.values()])
    kpath_xticks.append(idx_kpt['G'][1])
    kpath_xticks = sorted(kpath_xticks)
    ax.set_xticks(kpath_xticks)
    ax.set_xticklabels(kpath_labels)


    for n in range(len(kpath_xticks)):
        ax.axvline(x = kpath_xticks[n], linewidth = 1, color = 'k')

    
    ax.set_xlim(0, int_evals_038.shape[1] - 1)
    ax.set_ylabel(r"Band Energy - $E_F$ (eV)")
    ax.set_title("Comparing Minimal Models")
    ax.legend(["Full model", "Cutoff at 0.038 eV", "Cutoff at 0.05 eV", "Cutoff at 0.45 eV"])

# calling the function
lanthanum_w90_input = w90('La2CuO4 Files', 'lco')
compareBandStructure(lanthanum_w90_input, 12.7367)